# Publication post-processor

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('../output/progress.json', orient="records")
df.tail()

,dirname,title,authors,publisher,year,isbn,doi,llm_isbn,llm_doi,summary,cover_image
index,,,,,,,,,,,
129,/Volumes/SJF/CVI_OPACID_SJF_9788089313013,CVI_OPACID_SJF_9788089313013,[],,,,,,,,
130,/Volumes/SJF/CVI_OPACID_SJF_9788089313358,CVI_OPACID_SJF_9788089313358,[],,,,,,,,
131,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_1,CVI_OPACID_SJF_ZB_uloh_1,[],,,,,,,,
132,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_3,CVI_OPACID_SJF_ZB_uloh_3,[],,,,,,,,
133,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_4,CVI_OPACID_SJF_ZB_uloh_4,[],,,,,,,,


In [3]:
df['authors'] = df['authors'].apply(lambda x: '|'.join(x) if isinstance(x, list) else str(x))

In [4]:
df.dropna(subset=['authors']).to_json('../output/progress.json', orient="records")